In [89]:
import pyrealsense2.pyrealsense2 as rs
import numpy as np
import os
import cv2

directory = "framesets"
newfile = "Classroom_9_Roald_Print"

In [90]:
# If the given file already exists
if os.path.isfile(os.path.join(directory, newfile + ".bag")):
    print("File already exists!")
else:
    # Configure depth and color streams
    pipeline = rs.pipeline()
    saver = rs.save_single_frameset(filename = os.path.join(directory, newfile))
    colorizer = rs.colorizer()

    config = rs.config()
    config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
    config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

    # Get device product line for setting a supporting resolution
    pipeline_wrapper = rs.pipeline_wrapper(pipeline)
    pipeline_profile = config.resolve(pipeline_wrapper)
    device = pipeline_profile.get_device()
    device_product_line = str(device.get_info(rs.camera_info.product_line))

    # Start the pipeline
    profile = pipeline.start(config)

    # Print the depth scale
    depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()
    print("Depth Scale: {} meters".format(depth_scale))

    # Initialize the Haar-cascade classifier
    face_cascade = cv2.CascadeClassifier();
    face_cascade.load("ext/haarcascade_frontalface_default.xml");

    try:
        while True:
            # Wait for a coherent pair of frames: depth and color
            frames = pipeline.wait_for_frames()
            align = rs.align(rs.stream.color)
            aligned_frames = align.process(frames)
            depth_frame = aligned_frames.get_depth_frame().as_frame()
            depth_color_frame = rs.colorizer().colorize(depth_frame)
            color_frame = aligned_frames.get_color_frame().as_frame()
            if not depth_frame or not color_frame:
                continue

            # Convert images to numpy arrays
            depth_image = np.asanyarray(depth_frame.get_data())
            depth_colormap = np.asanyarray(depth_color_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())
            gray_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
            depth_colormap_dim = depth_colormap.shape
            color_colormap_dim = color_image.shape

            # Apply Haar-cascade classifier
            faces = face_cascade.detectMultiScale(gray_image)

            # Extract a copy of the original images to draw onto
            color_image_display = color_image.copy()
            depth_colormap_display = depth_colormap.copy()

            for (x,y,w,h) in faces:
                cv2.rectangle(color_image_display, (x, y), (x+w, y+h), (0, 0, 255), 1)
                cv2.rectangle(depth_colormap_display, (x, y), (x+w, y+h), (0, 0, 255), 1)
                extracted_face_color = color_image_display[y:y+h,x:x+w]
                extracted_face_depth = color_image_display[y:y+h,x:x+w]

            # If depth and color resolutions are different, resize color image to match depth image for display
            if depth_colormap_dim != color_colormap_dim:
                resized_color_image = cv2.resize(color_image_display, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
                images = np.hstack((resized_color_image, depth_colormap_display))
            else:
                images = np.hstack((color_image_display, depth_colormap_display))

            # Show images
            cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
            cv2.imshow('RealSense', images)

            key = cv2.waitKey(1)
            # Press esc or 'q' to close the image window
            if key & 0xFF == ord('q') or key == 27:
                saver.process(frames)

                # Remove the numbers that were appended to the filename
                filenames = [filename for filename in os.listdir(directory) if filename.startswith(newfile)]
                os.rename(os.path.join(directory, filenames[0]), os.path.join(directory, newfile + ".bag"))
                cv2.destroyAllWindows()
                break

    finally:

        # Stop streaming
        pipeline.stop()

Depth Scale: 0.0010000000474974513 meters
